In [ ]:
# on colab, you will have to install transformers and sentencepiece
%%bash
pip install transformers sentencepiece

In [ ]:
import numpy as np
import pandas as pd

import torch
import random

# huggingface
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


In [ ]:
# data logistics: load theses title and abstract
# limit_title_len=[4,10] restricts to titles in between 4 and 10 tokens
def load_thesis_data(path='res/theses.tsv', limit_title_len=None):
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html
    # limit to 'Sprache'==DE and title_len if necessary
    df = pd.read_csv(path, sep='\t')

    if limit_title_len:
        df['title_length'] = df['Titel'].apply(lambda x: len(x.split()))
        df = df[df['title_length'].between(limit_title_len[0], limit_title_len[1])] # pick your numbers or use std dev
    
    return df[df['Sprache'] == 'DE']


In [ ]:
# set up the models; they will download on first time use but this will take some time (1.2 GB)
# https://huggingface.co/transformers/model_doc/auto.html?highlight=autotokenizer#transformers.AutoTokenizer.from_pretrained
# https://huggingface.co/transformers/model_doc/auto.html?highlight=autotokenizer#transformers.AutoModelForSeq2SeqLM.from_pretrained

tokenizer = AutoTokenizer.from_pretrained("ml6team/mt5-small-german-finetune-mlsum")
model = AutoModelForSeq2SeqLM.from_pretrained("ml6team/mt5-small-german-finetune-mlsum")




In [ ]:
# method for summary generation, using the global model and tokenizer
def generate_summary(model, abstract, num_beams, repetition_penalty,
                    length_penalty, early_stopping, max_output_length):
    # source_encoding = tokenizer(...)
    source_encoding = tokenizer(
        abstract,
        max_length=784,  # this comes from the
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors="pt")

    # generated_ids = model.generate(...)
    generated_ids = model.generate(
        input_ids=source_encoding["input_ids"],
        attention_mask=source_encoding["attention_mask"],
        num_beams=num_beams,
        max_length=max_output_length,
        repetition_penalty=repetition_penalty,
        length_penalty=length_penalty,
        early_stopping=early_stopping,
        use_cache=True)

    # ...map to string using tokenizer.decode and return
    preds = [tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True) 
         for gen_id in generated_ids]

    return "".join(preds)


In [ ]:
# main program
df = load_thesis_data()
df.head()


In [ ]:
# now use the pre-trained model to generate some short summaries from the 
# abstracts, and compare them to the reference titles

# adjust these values as desired
num_beams = 2
repetition_penalty = 1.0
length_penalty = 2.0
max_output_length = 120

early_stopping = True

# sample from dataset, using abstracts as input to generate short summary (~title)
from IPython.display import HTML, display
def displaysum(summarize, generated, reference):
    display(HTML(f"""<table>
    <tr><td>summarize:</td><td>{summarize}</td></tr>
    <tr><td>generated:</td><td>{generated}</td></tr>
    <tr><td>reference:</td><td>{reference}</td></tr>
    </table>
    """))

for i in [random.randint(0, len(df) - 1) for _ in range(10)]:
    # load the values
    summarize = df.iloc[i].Abstract
    reference = df.iloc[i].Titel

    # generated = generate_summary(...)
    generated = generate_summary(model, "summarize: " + str(df.iloc[i].Abstract),
                                 num_beams, repetition_penalty,
                                 length_penalty, early_stopping, 120)

    displaysum(summarize, generated, reference)


In [ ]:
# Task 2: Fine-tuning

# As you could see, the summary quality is pretty much hit-or-miss. Let's use
# a good share of the data to fine-tune the pre-trained model to our task.

from torch.utils.data import Dataset

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device


In [ ]:
class ThesisDataset(Dataset):
    def __init__(self, df, tokenizer, max_input_len, max_output_len):
        self.tokenizer = tokenizer
        self.source_len = max_input_len
        self.summ_len = max_output_len
        self.Titel = df.Titel

        # T5 requires us to prepend the task
        self.Abstract = 'summarize: ' + df.Abstract

    def __len__(self):
        return len(self.Titel)

    def __getitem__(self, index):
        abstract = str(self.Abstract[index])
        title = str(self.Titel[index])

        # use tokenizer.batch_encode_plus to also get the masking
        source_tok = self.tokenizer.batch_encode_plus([abstract],
                                                      max_length=self.source_len, 
                                                      truncation=True,
                                                      pad_to_max_length=True,
                                                      return_tensors='pt')
        label_tok = self.tokenizer.batch_encode_plus([title],
                                                     max_length=self.summ_len,
                                                     truncation=True,
                                                     pad_to_max_length=True,
                                                     return_tensors='pt')

        input_ids = source_tok['input_ids'].squeeze()
        input_mask = source_tok['attention_mask'].squeeze()
        label_ids = label_tok['input_ids'].squeeze()
        label_mask = label_tok['attention_mask'].squeeze()

        return {
            'input_ids': input_ids.to(dtype=torch.long), 
            'input_mask': input_mask.to(dtype=torch.long), 
            'label_ids': label_ids.to(dtype=torch.long),
            'label_mask': label_mask.to(dtype=torch.long)
        }


In [ ]:

# for each point in the data loader, compute the forward pass, loss and
# backward pass

def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()

    for i, data in enumerate(loader, 0):
        y = data['label_ids'].to(device, dtype=torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()

        # set the padding symbols to -100 to be ignored by torch
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100

        inputs = data['input_ids'].to(device, dtype=torch.long)
        mask = data['input_mask'].to(device, dtype=torch.long)

        # compute forward pass
        outputs = model(input_ids=inputs, attention_mask=mask,
                        decoder_input_ids=y_ids, labels=lm_labels)

        loss = outputs[0]

        if i % 10 == 0:
            print({"Training Loss": loss.item()})

        # reset optimizer, do backwards pass and optimizer step    
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        


In [ ]:

# for validation, set the model to eval mode and compute all predictions
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for i, data in enumerate(loader, 0):
            y = data['label_ids'].to(device, dtype=torch.long)
            ids = data['input_ids'].to(device, dtype=torch.long)
            mask = data['input_mask'].to(device, dtype=torch.long)

            # make prediction
            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask,
                max_length=150,
                num_beams=2,
                repetition_penalty=2.5,
                length_penalty=1.0,
                early_stopping=True
                )
            
            # use tokenizer.decode to get predicted and target string
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            
            if i % 100 == 0:
                print(f'Completed {i}')

            predictions.extend(preds)
            actuals.extend(target)
    
    return predictions, actuals


In [ ]:

# defining some parameters that will be used later on in the training  
batch_size_train = 32
batch_size_vali = 4

max_input_len = 786    # 512?
max_output_len = 120

# set random seeds and deterministic pytorch for reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True

# TODO verify: tokenizer and df still loaded and current?

# split the dataframe into training and validation
df_train = df.sample(frac=0.8, random_state=seed)
df_vali = df.drop(df_train.index).reset_index(drop=True)
df_train = df_train.reset_index(drop=True)
print(f"df={df.shape}, train={df_train.shape}, vali={df_vali.shape}")

# Creating the Training and Validation dataset for further creation of Dataloader
ds_train = ThesisDataset(df_train, tokenizer, max_input_len, max_output_len)
ds_vali = ThesisDataset(df_vali, tokenizer, max_input_len, max_output_len)

# create data loaders for training and validation
from torch.utils.data import DataLoader
dl_train = DataLoader(ds_train, shuffle=True, num_workers=0, batch_size=batch_size_train)
dl_vali = DataLoader(ds_vali, shuffle=True, num_workers=0, batch_size=batch_size_vali)


In [ ]:

# we'll start from the same ml6team/mt5-small-german-finetune-mlsum that we
# used before in our baseline experiment; we will reload it below so that we
# maintain the base model
base = model

# this time, we'll load it explicitly as a T5ForConditionalGeneration; the
# tokenizer will be the same

from transformers import T5ForConditionalGeneration
model = T5ForConditionalGeneration.from_pretrained("ml6team/mt5-small-german-finetune-mlsum")
model = model.to(device)

# Defining the optimizer that will be used to tune the weights of the network in the training session. 
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-4)


In [ ]:
epochs_train = 3
epochs_vali = [1, 2, 3]

models = []

for epoch in range(epochs_train):
    # call the training routine from above
    train(epoch, tokenizer, model, device, dl_train, optimizer)

    if epoch in epochs_vali:
        # call the vali routine from above to generate some summaries
        predictions, actuals = validate(epoch, tokenizer, model, device, dl_vali)

        # display some...
        for i in [random.randint(0, len(predictions) - 1) for _ in range(10)]:
            displaysum(None, generated, reference)


In [ ]:
# load model and compare outputs
base = AutoModelForSeq2SeqLM.from_pretrained("ml6team/mt5-small-german-finetune-mlsum")
fine = model  # or any other checkpoint from res/mt5-small-fine-tune-...


In [ ]:
# pick some random theses and compare the two models
thesis_picks = [random.randint(0, len(df_vali) - 1) for _ in range(10)]
for num, i in enumerate(thesis_picks):
    print()
    # generate a summary with each of the models
    s0 = generate_summary(base, 'summarize: '+df_vali.iloc[i].Abstract,
                          num_beams, repetition_penalty,
                          length_penalty, early_stopping, max_output_len)
    s1 = generate_summary(base, ('summarize: '+df_vali.iloc[i].Abstract).lower(),
                          num_beams, repetition_penalty,
                          length_penalty, early_stopping, max_output_len)
    s2 = generate_summary(fine, ('summarize: '+df_vali.iloc[i].Abstract).lower(),
                          num_beams, repetition_penalty,
                          length_penalty, early_stopping, max_output_len)
    display(HTML(f"""<table>
    <tr><td>summarize:</td><td>{df_vali.iloc[i].Abstract}</td></tr>
    <tr><td>base:</td><td>{s0}</td></tr>
    <tr><td>base (lc):</td><td>{s1}</td></tr>
    <tr><td>fine:</td><td>{s2}</td></tr>
    <tr><td>reference:</td><td>{df_vali.iloc[i].Titel}</td></tr>
    </table>
    """))
